## Create gradient maps based on input location

This script allows you to create a gradient map of the available point clouds for a particular region of interest based on the topographical location of the cortical input of tracer data ([DOI: 10.25493/AJVE-B8G](https://search.kg.ebrains.eu/instances/27dd2588-f4da-406c-9d01-8569bf912b0e)) for the visualization in [Meshview](https://meshview.apps.hbp.eu/?atlas=ABA_Mouse_CCFv3_2017_25um). 

We use the following gradient colours:

<div>
    <img src="img/gradientColours.png" width="1040"/>
</div>


| Injection Sites  |
| ------------- | 
|<img src="img/nex-cko_injections.png" width="1020"/>  |


The cell below will create 6 JSON files, one file for each input gradient. The colour and the experiments for each input area have been predefined. Files are saved following the naming convention: Nex-cKO_[animal-group]_[gradient of segment].json (e.g. Nex-dataset_WT-controls_gradient1.json)

In [1]:
# Import packages
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path

## Load experiment details

In [2]:
# Data folder
data_folder = os.path.join("..", "data")

# Check if output folder exist, if not create the folder
output_folder = os.path.join("..", "output")
if not os.path.exists(output_folder):
   os.makedirs(output_folder)
   print("The new directory is created!")

# Load the experiment information
expInfo = pd.read_excel(os.path.join("..", "ExpOverview.xlsx"), sheet_name="Nex-cKO")

## Select Genotype of Interest

In [3]:
# First define the genotype of interest
listOfDatasets = ["WT-controls", "littermate-controls", "Nex-cKO", "all-cases"]
dataset = 0
while dataset not in ["1", "2", "3", "4"]:
    dataset = input("What is your region of interest? 1) WT, 2) littermate controls, 3) Nex-cKO transgenic mice, 4) all cases\n")
    if dataset not in ["1", "2", "3", "4"]:
        print("Please select a number between 1 and 4!\n")
    else:
        print(f"Selected dataset: {listOfDatasets[int(dataset)-1]}\n")

Selected dataset: all-cases



## Create the gradient files

In [4]:
gradient_color = [
    [250, 250, 110],
    [15, 223, 124],
    [74, 189, 140],
    [0, 150, 142],
    [1, 110, 124],
    [42, 72, 88]
]

for gradient_num in list(np.arange(1,7,1)):
    nameFile = f"gradient{gradient_num}"
    rgb = gradient_color[gradient_num-1]

    # Make subselection based on chosen genotype and gradient
    if dataset == "1":
        subset = expInfo[(expInfo.genotype == "C57BL/6J") & (expInfo.gradient == gradient_num)].reset_index(drop=True)
    elif dataset == "2":
        subset = expInfo[(expInfo.genotype == "Nex-cKO-Neg") & (expInfo.gradient == gradient_num)].reset_index(drop=True)
    elif dataset == "3":
        subset = expInfo[(expInfo.genotype == "Nex-cKO-Pos") & (expInfo.gradient == gradient_num)].reset_index(drop=True)
    else:
        subset = expInfo[expInfo.gradient == gradient_num].reset_index(drop=True)

    points_list = []
    for expName in subset.expName:
        fname = sorted(Path(data_folder).rglob(f"*{expName}*.json"))

        with open(fname[0], 'r') as f:
            points= json.load(f)
        
        points[0].update({"r": int(rgb[0]), "g": int(rgb[1]), "b": int(rgb[2])})
        points_list.append(points[0])

    # Saving the JSON file
    out_file = os.path.join(output_folder, f"Nex-dataset_{listOfDatasets[int(dataset)-1]}_{nameFile}.json")
    print(f"Saving file {out_file}")
    with open(out_file, 'w', encoding='utf-8') as fi:
        json.dump(points_list, fi, ensure_ascii=False)

print("Done!")


Saving file ..\output\Nex-dataset_all-cases_gradient1.json
Saving file ..\output\Nex-dataset_all-cases_gradient2.json
Saving file ..\output\Nex-dataset_all-cases_gradient3.json
Saving file ..\output\Nex-dataset_all-cases_gradient4.json
Saving file ..\output\Nex-dataset_all-cases_gradient5.json
Saving file ..\output\Nex-dataset_all-cases_gradient6.json
Done!
